<a href="https://colab.research.google.com/github/MajiroZ/for_git_study/blob/master/U_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##【問題1】学習・推定

In [27]:
!python main.py

2025-01-08 12:19:26.843747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-08 12:19:26.870412: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-08 12:19:26.878021: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-08 12:19:26.895161: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-08 12:19:28.344715: W tensorflow/compiler/tf2

##【問題2】コードリーディング

---
### U-Netコードリーディングの要点まとめ

#### 1. 中核部分
`model.py`でのU-Netモデルの重要部分

```python
up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2,2))(drop5))
merge6 = concatenate([drop4, up6], axis=3)
```

この部分では、`UpSampling2D`を使用して特徴マップの空間解像度を2倍にし、`concatenate`関数で下位層の特徴マップと結合する。これにより、U-Netのスキップ接続が実現される。これがU-Netの特徴であり、高精度なセグメンテーションを可能にしている。

---

#### 2. 使用ライブラリのインポート
```python
import glob
import cv2
import numpy as np
import skimage.io as io
import skimage.transform as trans
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)
```
Kerasを中心に、画像処理のために`cv2`や`skimage`を利用。`ModelCheckpoint`で学習中のモデルの保存も行う

---

#### 3. データの読み込み
```python
image_path_list = glob.glob("competition_data/train/images/*")
X_train, y_train = [], []

for image_path in image_path_list[:100]:
    image = cv2.resize(cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) / 255, (256, 256))
    X_train.append(image)
    mask = cv2.resize(cv2.imread(image_path.replace("images", "masks"), cv2.IMREAD_GRAYSCALE) / 255, (256, 256))
    y_train.append(mask)

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.expand_dims(X_train, axis=-1)
y_train = np.expand_dims(y_train, axis=-1)
```
1. 画像とマスクのパスをリスト化
2. グレースケール画像を正規化しつつリサイズ
3. `X_train`と`y_train`にそれぞれ画像とマスクを追加
4. `numpy.array`に変換して学習用の形に整形

---

#### 4. U-Netモデルの定義
```python
def unet(pretrained_weights=None, input_size=(256, 256, 1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    # 略 (他の層も同様に定義)

    conv10 = Conv2D(1, 1, activation='sigmoid', name="output")(conv9)
    model = Model(inputs=inputs, outputs=conv10)

    model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    
    if pretrained_weights:
        model.load_weights(pretrained_weights)

    return model
```
- U-Netモデルはエンコーダとデコーダの2つの部分で構成。
- スキップ接続により、エンコーダの特徴マップをデコーダ側で使用。
- 出力層はシグモイド活性化関数で、二値セグメンテーションに対応。

---

#### 5. モデルの学習と保存
```python
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss', verbose=1, save_best_only=True)
model = unet()
model.fit(X_train, y_train, batch_size=3, epochs=1, callbacks=[model_checkpoint])
```
学習途中で最良のモデルを`unet_membrane.hdf5`に保存

---

#### 6. 推論
```python
y_pred = model.predict(np.array([X_train[30, :, :, 0]]))
fig, ax = plt.subplots(1, 2)
ax[0].imshow(y_pred[0, :, :, 0], cmap='gray')
ax[1].imshow(y_train[30, :, :, 0], cmap='gray')
```
- `model.predict`で画像を入力し、出力マスクを取得。
- Matplotlibで予測結果と正解ラベルを並べて可視化。

---